# Determining the CDS Basket Spread

An analysis of pricing a CDS Basket 

## NOTE THAT THE API WILL CHANGE BUT THE UNDERLYING MODELS WILL REMAIN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.credit import *
from financepy.products.rates import *
from financepy.utils import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [3]:
value_date = Date(1, 8, 2007)
settle_date = value_date.add_weekdays(1)

## Build Ibor Curve

In [4]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
depo = IborDeposit(settle_date, "1D", 0.0502, dcType)
depos = [depo]
swap1 = IborSwap(settle_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
step_in_date = value_date.add_weekdays(0)
value_date = step_in_date

In [6]:
maturity3Y = value_date.next_cds_date(36)
maturity5Y = value_date.next_cds_date(60)
maturity7Y = value_date.next_cds_date(84)
maturity10Y = value_date.next_cds_date(120)

### Heterogeneous Curves

In [7]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_date, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [8]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40 

In [9]:
cdsIndex = CDSIndexPortfolio()

In [10]:
spd3Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity10Y, heteroIssuerCurves)

In [11]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678820972252545
Homogeneous curve 5Y: 35.53919235929797
Homogeneous curve 7Y: 49.011853495442594
Homogeneous curve 10Y: 61.41369871102576


In [12]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_date, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Baskets

In [13]:
num_credits = 5

In [14]:
issuer_curves = heteroIssuerCurves[0:num_credits]

In [15]:
basketMaturity = Date(20, 12, 2011)
cdsIndex = CDSIndexPortfolio()

In [16]:
intrinsicSpd = cdsIndex.intrinsic_spread(value_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("INTRINSIC SPD BASKET MATURITY", intrinsicSpd)

totalSpd = cdsIndex.total_spread(value_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("SUMMED UP SPD BASKET MATURITY", totalSpd)

minSpd = cdsIndex.min_spread(value_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("MINIMUM SPD BASKET MATURITY", minSpd)

maxSpd = cdsIndex.max_spread(value_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("MAXIMUM SPD BASKET MATURITY", maxSpd)

INTRINSIC SPD BASKET MATURITY 29.03716340808207
SUMMED UP SPD BASKET MATURITY 145.80584474439172
MINIMUM SPD BASKET MATURITY 9.62014416486713
MAXIMUM SPD BASKET MATURITY 73.20754781040397


In [17]:
basket = CDSBasket(value_date,basketMaturity)

## Gaussian Copula Model

WARNING: THE INTERFACE ON THIS FUNCTION WILL CHANGE SOON! THEY ALSO NEED TO BE ADAPTED TO USE NUMBA.

In [18]:
seed = 42
doF = 5
num_trials = 5000

print("NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10")
print("=========================================================")

for ntd in range(1, num_credits + 1):
    for beta in [0.0, 0.25, 0.5, 0.75, 0.90, 0.9999]:
        rho = beta * beta
        beta_vector = np.ones(num_credits) * beta
        corr_matrix = corr_matrix_generator(rho, num_credits)
        for num_trials in [5000]:
            v1 = basket.value_gaussian_mc(value_date,ntd,issuer_curves,corr_matrix,libor_curve,num_trials,seed)
            v2 = basket.value_1f_gaussian_homo(value_date,ntd,issuer_curves,beta_vector,libor_curve)
            v3 = basket.value_student_t_mc(value_date, ntd, issuer_curves, corr_matrix, doF, libor_curve, num_trials,seed)
            print("%7d  %5.2f    %d    %9.3f   %9.3f    %9.3f"% (num_trials, rho, ntd, v1[2] * 10000, v2[3] * 10000, v3[2] *10000))
    print("=========================================================")


NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10


IndexError: index 18 is out of bounds for axis 0 with size 18

Copyright (c) 2020 Dominic O'Kane